In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates. All rights reserved.

# Render a textured mesh

This tutorial shows how to:
- load a mesh and textures from an `.obj` file.
- set up a renderer
- render the mesh
- vary the rendering settings such as lighting and camera position
- use the batching features of the pytorch3d API to render the mesh from different viewpoints

## 0. Install and Import modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [2]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [3]:
import os
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    DirectionalLights,
    Materials,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    TexturesUV,
    TexturesVertex
)

# add path for demo utils functions
import sys
import os
sys.path.append(os.path.abspath(''))

If using **Google Colab**, fetch the utils file for plotting image grids:

In [4]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid

--2023-08-09 03:53:19--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘plot_image_grid.py’

plot_image_grid.py  100%[===================>]   1.57K  --.-KB/s    in 0s      

2023-08-09 03:53:19 (30.0 MB/s) - ‘plot_image_grid.py’ saved [1608/1608]



OR if running **locally** uncomment and run the following cell:

In [5]:
# from utils import image_grid

In [6]:
!ls

drive  plot_image_grid.py  __pycache__	sample_data


In [7]:
from google.colab import drive

drive.mount('/content/drive')

import os

os.chdir("/content/drive/MyDrive/Colab Notebooks")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!ls

 1_b.gif	  data
 2.1.gif	  deform_form.ipynb
 3_b.gif	  plotly_rendered_meshes.ipynb
'a1(1-2).ipynb'   rendering_generic_3d_representations.ipynb
'a1(3).ipynb'	  rendering_texture_obj.ipynb
'a1(4).ipynb'


In [9]:
if torch.cuda.is_available():
  device=torch.device("cuda:0")
else:
  device=torch.device("cpu")
  print("gpu is not available")


In [10]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [11]:
from torch.cuda import is_available

from pytorch3d.renderer.mesh.textures import TexturesAtlas
import torch

from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import(
    chamfer_distance,
    mesh_edge_loss,
    mesh_normal_consistency,
    mesh_laplacian_smoothing
)

from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    SoftPhongShader,
    MeshRenderer,
    MeshRasterizer,
    RasterizationSettings,
    TexturesAtlas,
    TexturesVertex,
    PointsRenderer,
    AlphaCompositor,
    NormWeightedCompositor,
    PointsRasterizationSettings,
    PointsRenderer,
    PulsarPointsRenderer,
    PointsRasterizer,
)

import numpy as np

from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

In [12]:
obj_file="./data/bridge/bridge_pointcloud.npz"

import numpy as np

point_cloud=np.load(obj_file, allow_pickle=True)

verts=torch.tensor(point_cloud['verts']).to(device)
rgb=torch.tensor(point_cloud['rgb']).to(device)

point_clouds=Pointclouds(points=[verts], features=[rgb])

R,T=look_at_view_transform(4,10,0)
cameras=FoVPerspectiveCameras(R=R, T=T, device=device)

raster_settings=PointsRasterizationSettings(
    image_size=512,
    radius=0.003,
    points_per_pixel=1
)
rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)

lights=PointLights(device=device, location=[[0,0,-3]])

renderer=PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor()
)

print(type(point_clouds))
rend=renderer(point_clouds)
# rend_cpu=rend[0, ... , :3].cpu().numpy()



<class 'pytorch3d.structures.pointclouds.Pointclouds'>


In [13]:
plot_scene({
    "Pointcloud":{
        "bridge": point_clouds
    }
})